# House Prices Prediction

In [1]:
import numpy as np #linear algebra
import pandas as pd #data processing
import sklearn #machine learning
import matplotlib.pyplot as plt #visulization
import seaborn as sns; sns.set() #visulization
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.columns.values

array(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'Pav

The heatmap is a pretty good visualization of how the features relate. Next, let's look at just the SalePrice feature since that is what we are trying to predict.

In [4]:
train['OverallQual'] = train['OverallQual'].fillna(train['OverallQual'].mean())
train['GrLivArea'] = train['GrLivArea'].fillna(train['GrLivArea'].mean())
train['GarageArea'] = train['GarageArea'].fillna(train['GarageArea'].mean())
test['OverallQual'] = test['OverallQual'].fillna(test['OverallQual'].mean())
test['GrLivArea'] = test['GrLivArea'].fillna(test['GrLivArea'].mean())
test['GarageArea'] = test['GarageArea'].fillna(test['GarageArea'].mean())

train_features = train[['OverallQual', 'GrLivArea', 'GarageArea']]
test_features = test[['OverallQual', 'GrLivArea', 'GarageArea']]



In [5]:
X = np.array(train[['OverallQual', 'GrLivArea', 'GarageArea']])
y = np.array(train['SalePrice'])
X_test = np.array(test_features)

from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

n_cols = X.shape[1]

def create_model(dropout_rate=0.0, weight_constraint=0, neurons=1, network_depth=1, activation='relu'):
    #create model
    model = Sequential()
    for depth in range(network_depth):
        model.add(Dense(units=neurons, input_shape=(n_cols,), activation='relu',
                        kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dense(units=neurons, activation=activation))
        model.add(Dropout(dropout_rate))
        
    model.add(Dense(units=1, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, epochs=10, 
                             batch_size=50, verbose=0)

#activation
activation = ['relu', 'elu', 'linear', 'selu']

#depth
network_depth = [6,3,1]

#dropout 
dropout_rate = [0.25, 0.50, 0.75]

#weight contraint
weight_constraint = [1, 2, 3, 4, 5]

#neurons: width of the neural net
neurons = [32, 64, 128]

#Parameters desired to test in the search
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint, neurons=neurons, network_depth=network_depth,
                 activation=activation)

#Initialization of GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    n_jobs=1, scoring='accuracy', verbose=3)

#sets the results of GridSearchCV to variable
grid_result = grid.fit(X, y)


#prints best result
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
y_predict = grid.predict(X_predict)   

submit = pd.DataFrame({'Id' : test.loc[:,'Id'],
                       #'Survived': surv_pred.T})
                       'SalePrice': y_predict.T})
submit.to_csv("submit.csv", index=False)


Fitting 3 folds for each of 540 candidates, totalling 1620 fits
[CV] activation=relu, dropout_rate=0.25, network_depth=6, neurons=32, weight_constraint=1 


InternalError: Blas GEMM launch failed : a.shape=(50, 3), b.shape=(3, 32), m=50, n=32, k=3
	 [[Node: dense_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_dense_1_input_0/_53, dense_1/kernel/read)]]
	 [[Node: metrics/acc/Mean_1/_87 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2493_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dense_1/MatMul', defined at:
  File "C:\Users\wrice\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\wrice\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\wrice\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\wrice\Anaconda3\

In [ ]:
submit = pd.DataFrame({'Id' : test.loc[:,'Id'],
                       'SalePrice': y_predict.T})
submit.to_csv("submit.csv", index=False)